# Aggressive Regularization Experiment

**CRITICAL PIVOT**: The CV-LB gap is STRUCTURAL (not procedural). GroupKFold(5) CV was only 1.13x higher than LOO CV, not 3-5x expected.

**Hypothesis**: The gap is due to overfitting to the training distribution. Stronger regularization should reduce the gap, even if CV gets worse.

**Implementation**:
- MLP: Dropout 0.2 → 0.5, weight decay 1e-4 → 1e-3, hidden [128,128] → [32,16], epochs 150 → 100
- LGBM: max_depth 6 → 3, min_child_samples 10 → 20, reg_alpha/lambda 0.1 → 1.0, n_estimators 200 → 100
- GP: length_scale 1.0 → 2.0, noise_level 0.1 → 0.5

**Expected outcome**:
- CV will get WORSE (maybe 0.012-0.015)
- But LB might IMPROVE if the gap is due to overfitting
- If LB improves relative to CV, this confirms the overfitting hypothesis

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer (for MLP and LGBM) - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')

Full feature dimension: 145


In [5]:
# Simple Featurizer (for GP) - 18 features (Spange + Arrhenius kinetics)
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Simple feature dimension (for GP): {SimpleFeaturizer().feats_dim}')

Simple feature dimension (for GP): 18


In [6]:
# AGGRESSIVE REGULARIZATION: Smaller MLP with heavy dropout
class RegularizedMLP(nn.Module):
    """Smaller MLP with aggressive regularization.
    
    Changes from original:
    - Hidden: [128, 128] -> [32, 16] (much smaller)
    - Dropout: 0.2 -> 0.5 (2.5x increase)
    - Uses LayerNorm for stability
    """
    def __init__(self, input_dim, output_dim=3, dropout=0.5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.LayerNorm(32),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(32, 16),
            nn.LayerNorm(16),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(16, output_dim)
        )
        
    def forward(self, x):
        return self.net(x)

print('RegularizedMLP defined: [32, 16] hidden, 0.5 dropout')

RegularizedMLP defined: [32, 16] hidden, 0.5 dropout


In [7]:
# AGGRESSIVE REGULARIZATION: GP + MLP + LGBM Ensemble
class RegularizedEnsemble:
    """Ensemble with aggressive regularization.
    
    Changes from original:
    - MLP: [128,128] -> [32,16], dropout 0.2 -> 0.5, weight_decay 1e-4 -> 1e-3, epochs 150 -> 100
    - LGBM: max_depth 6 -> 3, min_child_samples 10 -> 20, reg_alpha/lambda 0.1 -> 1.0, n_estimators 200 -> 100
    - GP: length_scale 1.0 -> 2.0, noise_level 0.1 -> 0.5
    """
    def __init__(self, data='single', gp_weight=0.15, mlp_weight=0.55, lgbm_weight=0.3):
        self.data_type = data
        self.gp_weight = gp_weight
        self.mlp_weight = mlp_weight
        self.lgbm_weight = lgbm_weight
        
        # Featurizers
        self.full_featurizer = FullFeaturizer(mixed=(data == 'full'))
        self.simple_featurizer = SimpleFeaturizer(mixed=(data == 'full'))
        
        # Scalers
        self.full_scaler = StandardScaler()
        self.simple_scaler = StandardScaler()
        self.y_scaler = StandardScaler()
        
        # Models - REGULARIZED
        self.mlp = RegularizedMLP(self.full_featurizer.feats_dim, output_dim=3, dropout=0.5).to(device)
        self.gps = []  # One GP per target
        self.lgbm_models = []  # One LGBM per target
        
    def train_model(self, X, Y):
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        # Scale
        X_full_scaled = self.full_scaler.fit_transform(X_full)
        X_simple_scaled = self.simple_scaler.fit_transform(X_simple)
        Y_scaled = self.y_scaler.fit_transform(Y_np)
        
        # Train MLP with AGGRESSIVE regularization
        self._train_mlp(X_full_scaled, Y_scaled)
        
        # Train GPs with LARGER length scale and noise
        self.gps = []
        for i in range(3):
            # AGGRESSIVE: length_scale 1.0 -> 2.0, noise_level 0.1 -> 0.5
            kernel = ConstantKernel(1.0) * Matern(length_scale=2.0, nu=2.5) + WhiteKernel(noise_level=0.5)
            gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=3, normalize_y=True, random_state=42)
            gp.fit(X_simple_scaled, Y_scaled[:, i])
            self.gps.append(gp)
        
        # Train LGBM with AGGRESSIVE regularization
        self.lgbm_models = []
        for i in range(3):
            lgbm_model = lgb.LGBMRegressor(
                n_estimators=100,  # 200 -> 100
                learning_rate=0.05,
                max_depth=3,  # 6 -> 3
                num_leaves=15,  # 31 -> 15
                min_child_samples=20,  # 10 -> 20
                reg_alpha=1.0,  # 0.1 -> 1.0
                reg_lambda=1.0,  # 0.1 -> 1.0
                random_state=42,
                verbose=-1
            )
            lgbm_model.fit(X_full_scaled, Y_scaled[:, i])
            self.lgbm_models.append(lgbm_model)
    
    def _train_mlp(self, X_scaled, Y_scaled):
        X_tensor = torch.tensor(X_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_scaled, dtype=torch.double).to(device)
        
        dataset = TensorDataset(X_tensor, Y_tensor)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)
        
        # AGGRESSIVE: weight_decay 1e-4 -> 1e-3
        optimizer = torch.optim.AdamW(self.mlp.parameters(), lr=1e-3, weight_decay=1e-3)
        # AGGRESSIVE: epochs 150 -> 100
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)
        
        # Weighted MSE loss
        target_weights = torch.tensor([1.0, 1.0, 2.0], dtype=torch.double).to(device)
        
        self.mlp.train()
        for epoch in range(100):  # 150 -> 100
            for X_batch, Y_batch in loader:
                optimizer.zero_grad()
                pred = self.mlp(X_batch)
                loss = ((pred - Y_batch) ** 2 * target_weights).mean()
                loss.backward()
                optimizer.step()
            scheduler.step()
    
    def predict(self, X):
        # Featurize
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        # Scale
        X_full_scaled = self.full_scaler.transform(X_full)
        X_simple_scaled = self.simple_scaler.transform(X_simple)
        
        # MLP prediction
        self.mlp.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
            mlp_pred_scaled = self.mlp(X_tensor).cpu().numpy()
        
        # GP prediction
        gp_pred_scaled = np.column_stack([gp.predict(X_simple_scaled) for gp in self.gps])
        
        # LGBM prediction
        lgbm_pred_scaled = np.column_stack([lgbm.predict(X_full_scaled) for lgbm in self.lgbm_models])
        
        # Ensemble (weighted average in scaled space)
        ensemble_pred_scaled = (
            self.gp_weight * gp_pred_scaled + 
            self.mlp_weight * mlp_pred_scaled + 
            self.lgbm_weight * lgbm_pred_scaled
        )
        
        # Inverse transform
        ensemble_pred = self.y_scaler.inverse_transform(ensemble_pred_scaled)
        
        # Clip to valid range
        ensemble_pred = np.clip(ensemble_pred, 0, 1)
        
        return torch.tensor(ensemble_pred)
    
    def predict_with_tta(self, X):
        """Predict with test-time augmentation for mixed solvents."""
        if self.data_type == 'single':
            return self.predict(X)
        
        # For mixed solvents, average predictions from both orderings
        pred1 = self.predict(X)
        
        # Create flipped version
        X_flip = X.copy()
        X_flip["SOLVENT A NAME"] = X["SOLVENT B NAME"]
        X_flip["SOLVENT B NAME"] = X["SOLVENT A NAME"]
        X_flip["SolventB%"] = 1 - X["SolventB%"]
        
        # Featurize with flip=True
        X_full_flip = self.full_featurizer.featurize(X_flip, flip=False)
        X_simple_flip = self.simple_featurizer.featurize(X_flip, flip=False)
        
        X_full_scaled = self.full_scaler.transform(X_full_flip)
        X_simple_scaled = self.simple_scaler.transform(X_simple_flip)
        
        self.mlp.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
            mlp_pred_scaled = self.mlp(X_tensor).cpu().numpy()
        
        gp_pred_scaled = np.column_stack([gp.predict(X_simple_scaled) for gp in self.gps])
        lgbm_pred_scaled = np.column_stack([lgbm.predict(X_full_scaled) for lgbm in self.lgbm_models])
        
        ensemble_pred_scaled = (
            self.gp_weight * gp_pred_scaled + 
            self.mlp_weight * mlp_pred_scaled + 
            self.lgbm_weight * lgbm_pred_scaled
        )
        
        pred2 = self.y_scaler.inverse_transform(ensemble_pred_scaled)
        pred2 = np.clip(pred2, 0, 1)
        pred2 = torch.tensor(pred2)
        
        # Average
        return (pred1 + pred2) / 2

print('RegularizedEnsemble defined with AGGRESSIVE regularization:')
print('  MLP: [32,16] hidden, 0.5 dropout, 1e-3 weight_decay, 100 epochs')
print('  LGBM: max_depth=3, min_child_samples=20, reg_alpha/lambda=1.0, n_estimators=100')
print('  GP: length_scale=2.0, noise_level=0.5')

RegularizedEnsemble defined with AGGRESSIVE regularization:
  MLP: [32,16] hidden, 0.5 dropout, 1e-3 weight_decay, 100 epochs
  LGBM: max_depth=3, min_child_samples=20, reg_alpha/lambda=1.0, n_estimators=100
  GP: length_scale=2.0, noise_level=0.5


In [8]:
# Test the model
X_single, Y_single = load_data("single_solvent")
print(f"Single solvent data: {X_single.shape[0]} samples, {len(X_single['SOLVENT NAME'].unique())} unique solvents")

X_full, Y_full = load_data("full")
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
print(f"Full data: {X_full.shape[0]} samples, {len(ramps)} unique ramps")

Single solvent data: 656 samples, 24 unique solvents
Full data: 1227 samples, 13 unique ramps


In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = RegularizedEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:21, 21.60s/it]

2it [00:46, 23.61s/it]

3it [01:05, 21.31s/it]

4it [01:22, 19.92s/it]

...

21it [08:51, 25.94s/it]

22it [09:15, 25.51s/it]

23it [09:43, 26.35s/it]

24it [10:08, 25.82s/it]

24it [10:08, 25.35s/it]

In [10]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = RegularizedEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict_with_tta(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [01:03, 63.09s/it]

2it [02:06, 63.51s/it]

3it [03:08, 62.53s/it]

4it [04:12, 63.03s/it]

...

10it [10:37, 65.05s/it]

11it [11:45, 65.96s/it]

12it [12:53, 66.76s/it]

13it [13:57, 65.71s/it]

13it [13:57, 64.40s/it]

In [11]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [12]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== AGGRESSIVE REGULARIZATION CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')

print(f'\n=== COMPARISON ===')
print(f'exp_041 baseline (GP 0.15 + MLP 0.55 + LGBM 0.3): CV 0.008199')
print(f'exp_043 (AGGRESSIVE REGULARIZATION): CV {overall_mse:.6f}')

if overall_mse < 0.008199:
    improvement = (0.008199 - overall_mse) / 0.008199 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_041!')
else:
    degradation = (overall_mse - 0.008199) / 0.008199 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_041')
    print(f'\nBUT THIS IS EXPECTED! The hypothesis is that worse CV might give better LB.')
    print(f'If the CV-LB gap is due to overfitting, this model should have better LB.')
    
    # Estimate LB using old relationship
    estimated_lb = 4.30 * overall_mse + 0.0524
    print(f'\nEstimated LB (using old relationship): {estimated_lb:.4f}')
    print(f'Best LB so far: 0.0877')
    print(f'Target: 0.0347')
    print(f'\nIf aggressive regularization reduces the CV-LB gap, actual LB might be BETTER than estimated.')


=== AGGRESSIVE REGULARIZATION CV SCORE ===
Single Solvent MSE: 0.008916 (n=656)
Full Data MSE: 0.009048 (n=1227)
Overall MSE: 0.009002

=== COMPARISON ===
exp_041 baseline (GP 0.15 + MLP 0.55 + LGBM 0.3): CV 0.008199
exp_043 (AGGRESSIVE REGULARIZATION): CV 0.009002

✗ WORSE: 9.79% worse than exp_041

BUT THIS IS EXPECTED! The hypothesis is that worse CV might give better LB.
If the CV-LB gap is due to overfitting, this model should have better LB.

Estimated LB (using old relationship): 0.0911
Best LB so far: 0.0877
Target: 0.0347

If aggressive regularization reduces the CV-LB gap, actual LB might be BETTER than estimated.
